<a href="https://colab.research.google.com/github/Ramanarayanan/DeepLearningAndGenerativeAi/blob/main/Copy_of_Accenture_results_pdf_lama_chat_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install langchain langchain_groq langchain-community unstructured InstructorEmbedding sentence_transformers langchain-huggingface install faiss-cpu langchain-community

  Using cached unstructured-0.16.11-py3-none-any.whl.metadata (24 kB)
  Using cached InstructorEmbedding-1.0.1-py2.py3-none-any.whl.metadata (20 kB)
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [34]:
!pip install -U langchain-community pymupdf rapidocr-onnxruntime faiss-cpu langchain_groq langchain-huggingface

In [35]:
import os

import pickle
import time
import langchain

from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from google.colab import files

In [37]:
uploaded = files.upload()

if uploaded:
  file_name = list(uploaded.keys())[0]

else:
  print("No file uploaded")
  exit

Saving accenture.pdf to accenture (1).pdf


In [38]:
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader

loader = PyMuPDFLoader(file_name, extract_images=True)
Filedata = loader.load()

In [39]:
# text splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

convertedDocs = text_splitter.split_documents(Filedata)
print(convertedDocs)

[Document(metadata={'source': 'accenture (1).pdf', 'file_path': 'accenture (1).pdf', 'page': 0, 'total_pages': 11, 'format': 'PDF 1.6', 'title': 'FY25 Q1 Earnings 8-K Exhibit', 'author': 'anonymous', 'subject': '', 'keywords': '', 'creator': 'Workiva', 'producer': 'Wdesk Fidelity Content Translations Version 010.008.040', 'creationDate': 'D:20241218201405Z', 'modDate': "D:20241219115942-05'00'", 'trapped': ''}, page_content='Accenture Reports First-Quarter Fiscal 2025 Results\nAccenture’s strong Q1 FY25 results reflect broad-based revenue growth across all \nmarkets and industry groups; Company raises fiscal 2025 revenue outlook\nNew bookings of $18.7 billion, an increase of 1% in both U.S. dollars and local currency  \nGenerative AI new bookings of $1.2 billion\nRevenues of $17.7 billion, an increase of 9% in U.S. dollars and 8% in local currency, with \nconsulting revenues of $9.0 billion and managed services revenues of $8.6 billion\nGAAP operating margin of 16.7%, an increase of 90

In [40]:
def remove_ws(Filedocument):
    text = Filedocument.page_content.replace('\n','')
    Filedocument.page_content = text
    return Filedocument

# applied on the docs
docs = [remove_ws(Filedocument) for Filedocument in convertedDocs]

In [41]:
from google.colab import userdata
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0.9, max_tokens=500,
    groq_api_key=userdata.get('groq_api_key_valu'),
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("Explain about this model and cutt off date ")
print(response.content)

I'm referring to my language model, which is a type of artificial intelligence (AI) designed to process and generate human-like text.

**Model Explanation:**
My model is a transformer-based language model, specifically a type of recurrent neural network (RNN) designed for natural language processing (NLP) tasks. It's trained on a massive dataset of text from various sources, including books, articles, and online content. This training data allows me to learn patterns, relationships, and structures of language, enabling me to generate human-like text.

**Key Features:**

1. **Language Understanding:** I can comprehend and interpret human language, including nuances, context, and subtleties.
2. **Text Generation:** I can create text based on a given prompt, topic, or style, using my understanding of language patterns and structures.
3. **Knowledge Retrieval:** I have been trained on a vast amount of text data, allowing me to retrieve and provide information on a wide range of topics.

**

In [42]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [43]:
from langchain.vectorstores import FAISS
vectorindex_Lama = FAISS.from_documents(docs, embeddings)

In [44]:
# Storing vector index create in local
file_path="vector_database_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_Lama, f)

In [45]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [46]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())



In [47]:
query = "Explain financial review of accenture, new bookings,Revenues by Geographic Market,Revenues by Industry Group,Dividend in detail and show in table format "
# query = "what are the main features of punch iCNG?"

# langchain.debug=True

response = chain({"question": query})

In [50]:
while True:
  user_question = input("Enter your question (or type 'exit'): ")
  if user_question.lower() == "exit":
    break

  answer = response = chain({"question": user_question})
  print(answer.get('answer'))

KeyboardInterrupt: Interrupted by user

In [49]:
print(response.get('answer'))

The available information provides some details about Accenture's financial performance, including new bookings, revenues, and dividend. However, it lacks specific data on Revenues by Geographic Market and Revenues by Industry Group. Here is a summary of the available information:

New bookings: $18.7 billion, with a 1% increase in both U.S. dollars and local currency.
Revenues: $17.7 billion, with a 9% increase in U.S. dollars and 8% in local currency.
Consulting revenues: $9.0 billion.
Managed services revenues: $8.6 billion.
Quarterly cash dividend: $1.48 per share, with a 15% increase.

Here is a table summarizing some of the available information:

| **Category** | **Value** | **Growth** |
| --- | --- | --- |
| New Bookings | $18.7 billion | 1% (U.S. dollars and local currency) |
| Revenues | $17.7 billion | 9% (U.S. dollars), 8% (local currency) |
| Consulting Revenues | $9.0 billion | - |
| Managed Services Revenues | $8.6 billion | - |
| Quarterly Cash Dividend | $1.48 per shar

In [ ]:
query = "show the accenture Summary of Revenues in table format in detail  "



In [ ]:
result = chain({"question": query}).get('answer')

In [ ]:
print(result)